# COVID-19 Journal Classification using Machine Learning Techniques<br>
## Team Members:<br>
1.Yerramaddu Jahnavi - 181CO260 <br>
2.Swathi J S - 181CO155

The main aim of our system is to classify the abstracts related to covid with their respective journals so that a researcher can refer to articles of his interest from the required journals instead of searching all the articles.

### Importing the necessary libraries 

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import future

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize

from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

### Data Description | Loading the dataset 

In [62]:
data = pd.read_csv("metadata.csv")

In [63]:
data.shape

(490904, 19)

In [64]:
data.head(10)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
5,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/b2897e1277f56641193a6...,document_parses/pmc_json/PMC125340.xml.json,http://europepmc.org/articles/pmc125340?pdf=re...,NaN
6,5yhe786e,3bb07ea10432f7738413dff9816809cc90f03f99,PMC,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,PMC137267,11299062,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/3bb07ea10432f7738413d...,document_parses/pmc_json/PMC137267.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
7,8zchiykl,5806726a24dc91de3954001effbdffd7a82d54e2,PMC,The 21st International Symposium on Intensive ...,10.1186/cc1013,PMC137274,11353930,no-cc,The 21st International Symposium on Intensive ...,2001-05-02,"Ball, Jonathan; Venn, Richard",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/5806726a24dc91de39540...,document_parses/pmc_json/PMC137274.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
8,8qnrcgnk,faaf1022ccfe93b032c5608097a53543ba24aedb,PMC,Heme oxygenase-1 and carbon monoxide in pulmon...,10.1186/1465-9921-4-7,PMC193681,12964953,no-cc,"Heme oxygenase-1 (HO-1), an inducible stress p...",2003-08-07,"Slebos, Dirk-Jan; Ryter, Stefan W; Choi, Augus...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/faaf1022ccfe93b032c56...,document_parses/pmc_json/PMC193681.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
9,jg13scgo,5b44feca5d6ffaaeb6

In [65]:
data['source_x'].value_counts()

WHO                                    195986
Medline                                 95739
Medline; PMC                            80474
PMC                                     46072
Elsevier; Medline; PMC                  41022
MedRxiv; WHO                             9484
Elsevier; PMC                            6863
ArXiv                                    6015
BioRxiv; WHO                             2210
BioRxiv                                  1471
MedRxiv                                  1140
Medline; PMC; WHO                         931
MedRxiv; Medline; PMC; WHO                586
BioRxiv; Medline; PMC; WHO                396
Elsevier                                  380
Elsevier; Medline; PMC; WHO               305
MedRxiv; Medline; WHO                     289
BioRxiv; MedRxiv                          246
BioRxiv; Medline; WHO                     243
ArXiv; Medline; PMC                       225
Medline; WHO                              220
ArXiv; Medline                    

In [66]:
data.dtypes

cord_uid             object
sha                  object
source_x             object
title                object
doi                  object
pmcid                object
pubmed_id            object
license              object
abstract             object
publish_time         object
authors              object
journal              object
mag_id              float64
who_covidence_id     object
arxiv_id             object
pdf_json_files       object
pmc_json_files       object
url                  object
s2_id               float64
dtype: object

In [67]:
data['journal'].value_counts()

bioRxiv                                                      4773
PLoS One                                                     4398
BMJ                                                          4246
Sci Rep                                                      2057
Nature                                                       2015
                                                             ... 
Medicina oral, patologia oral y cirugia bucal                   1
ICT Express                                                     1
Australasian Journal of Educational Technology                  1
Le Journal medical libanais. The Lebanese medical journal       1
Eye Contact Lens                                                1
Name: journal, Length: 34523, dtype: int64

In [68]:
filt = (data['journal'] =='bioRxiv') | (data['journal'] == 'PLoS One') | (data['journal'] == 'BMJ')| (data['journal'] == 'Sci Rep') | (data['journal'] == 'Nature')
updated_df = data.loc[filt,['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id', 'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files', 'url', 's2_id']]
updated_df.to_csv('./data1.csv', index = False)


In [69]:
data_filtered=pd.read_csv("data1.csv")

In [70]:
data_filtered.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,rr2wwvue,c64f525d77bf8b2ae4439a875f1bd8eb197a0519,PMC,Association and Host Selectivity in Multi-Host...,10.1371/journal.pone.0000041,PMC1762347,17183670.0,cc-by,The distribution of multi-host pathogens over ...,2006-12-20,"Malpica, José M.; Sacristán, Soledad; Fraile, ...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/c64f525d77bf8b2ae4439...,document_parses/pmc_json/PMC1762347.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,"Klinkenberg, Don; Fraser, Christophe; Heesterb...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,x2zq0lnt,d882f79283751d6c7f2e48cb991ee8abbcd913ae,PMC,The Transmissibility of Highly Pathogenic Avia...,10.1371/journal.pone.0000349,PMC1831494,17406673.0,cc-by,BACKGROUND: With the increased occurrence of o...,2007-04-04,"Garske, Tini; Clarke, Paul; Ghani, Azra C.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/d882f79283751d6c7f2e4...,document_parses/pmc_json/PMC1831494.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
4,s4y6uxsb,305cf1019ca9c81126b2e527fedf79969c2c522f,PMC,"ElaD, a Deubiquitinating Protease Expressed by...",10.1371/journal.pone.0000381,PMC1847702,17440617.0,cc-by,BACKGROUND: Ubiquitin and ubiquitin-like prote...,2007-04-18,"Catic, André; Misaghi, Shahram; Korbel, Gregor...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/305cf1019ca9c81126b2e...,document_parses/pmc_json/PMC1847702.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN


In [71]:
data_filtered['journal'].value_counts()

bioRxiv     4773
PLoS One    4398
BMJ         4246
Sci Rep     2057
Nature      2015
Name: journal, dtype: int64

In [72]:
data_filtered.shape

(17489, 19)

### Data Preprocessing 
The following text preprocessing is implemented to convert the raw reviews to cleaned review, so that it will be easier for us to do feature extraction in the next step.

- remove html tags using BeautifulSoup
- remove non-character such as digits and symbols
- convert to lower case
- remove stop words such as "the" and "and" if needed
- convert to root words by stemming if needed


In [27]:
def CleanText(raw_text, remove_stopwords=False, stemming=False, split_text=False):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'lxml').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case 
    
    if remove_stopwords: # remove stopword
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    if stemming==True: # stemming
        #stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english') 
        words = [stemmer.stem(w) for w in words]
        
    if split_text==True:  # split text
        return (words)
    
    return( " ".join(words))

In [31]:
data_filtered.dtypes

cord_uid             object
sha                  object
source_x             object
title                object
doi                  object
pmcid                object
pubmed_id           float64
license              object
abstract             object
publish_time         object
authors              object
journal              object
mag_id              float64
who_covidence_id     object
arxiv_id            float64
pdf_json_files       object
pmc_json_files       object
url                  object
s2_id               float64
dtype: object

In [32]:
import warnings
warnings.filterwarnings("ignore")
cleaned_abstract=[]
for i in data_filtered['title']:
    cleaned_abstract.append(CleanText(i))
data_filtered['Cleaned title']=cleaned_abstract
data_filtered.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,Cleaned title
0,rr2wwvue,c64f525d77bf8b2ae4439a875f1bd8eb197a0519,PMC,Association and Host Selectivity in Multi-Host...,10.1371/journal.pone.0000041,PMC1762347,17183670.0,cc-by,The distribution of multi-host pathogens over ...,2006-12-20,"Malpica, José M.; Sacristán, Soledad; Fraile, ...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/c64f525d77bf8b2ae4439...,document_parses/pmc_json/PMC1762347.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,association and host selectivity in multi host...
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,"Klinkenberg, Don; Fraser, Christophe; Heesterb...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,the effectiveness of contact tracing in emergi...
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,the waiting time for inter country spread of p...
3,x2zq0lnt,d882f79283751d6c7f2e48cb991ee8abbcd913ae,PMC,The Transmissibility of Highly Pathogenic Avia...,10.1371/journal.pone.0000349,PMC1831494,17406673.0,cc-by,BACKGROUND: With the increased occurrence of o...,2007-04-04,"Garske, Tini; Clarke, Paul; Ghani, Azra C.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/d882f79283751d6c7f2e4...,document_parses/pmc_json/PMC1831494.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,the transmissibility of highly pathogenic avia...
4,s4y6uxsb,305cf1019ca9c81126b2e527fedf79969c2c522f,PMC,"ElaD, a Deubiquitinating Protease Expressed by...",10.1371/journal.pone.0000381,PMC1847702,17440617.0,cc-by,BACKGROUND: Ubiquitin and ubiquitin-like prote...,2007-04-18,"Catic, André; Misaghi, Shahram; Korbel, Gregor...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/305cf1019ca9c81126b2e...,document_parses/pmc_json/PMC1847702.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,elad a deubiquitinating protease expressed by ...


In [47]:
warnings.filterwarnings("ignore")
cleaned_abstract=[]
for i in data_filtered['abstract']:
    cleaned_abstract.append(CleanText(i))
data_filtered['Cleaned Abstract']=cleaned_abstract
data_filtered.head()

TypeError: object of type 'float' has no len()

### Language Detection Module

In [73]:
from langdetect import detect
language = []
for i in data_filtered['title']:
    language.append(detect(i))
data_filtered['Title Language']=language

In [74]:
data_filtered.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,Title Language
0,rr2wwvue,c64f525d77bf8b2ae4439a875f1bd8eb197a0519,PMC,Association and Host Selectivity in Multi-Host...,10.1371/journal.pone.0000041,PMC1762347,17183670.0,cc-by,The distribution of multi-host pathogens over ...,2006-12-20,"Malpica, José M.; Sacristán, Soledad; Fraile, ...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/c64f525d77bf8b2ae4439...,document_parses/pmc_json/PMC1762347.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,"Klinkenberg, Don; Fraser, Christophe; Heesterb...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en
3,x2zq0lnt,d882f79283751d6c7f2e48cb991ee8abbcd913ae,PMC,The Transmissibility of Highly Pathogenic Avia...,10.1371/journal.pone.0000349,PMC1831494,17406673.0,cc-by,BACKGROUND: With the increased occurrence of o...,2007-04-04,"Garske, Tini; Clarke, Paul; Ghani, Azra C.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/d882f79283751d6c7f2e4...,document_parses/pmc_json/PMC1831494.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en
4,s4y6uxsb,305cf1019ca9c81126b2e527fedf79969c2c522f,PMC,"ElaD, a Deubiquitinating Protease Expressed by...",10.1371/journal.pone.0000381,PMC1847702,17440617.0,cc-by,BACKGROUND: Ubiquitin and ubiquitin-like prote...,2007-04-18,"Catic, André; Misaghi, Shahram; Korbel, Gregor...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/305cf1019ca9c81126b2e...,document_parses/pmc_json/PMC1847702.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en


In [75]:
data_filtered['Title Language'].value_counts()

en    17194
it       84
ca       53
fr       46
ro       25
nl       13
es       12
da        9
de        9
pt        8
tl        7
af        6
et        4
so        3
cy        3
sw        2
hr        2
id        2
fi        2
vi        1
no        1
sv        1
tr        1
lt        1
Name: Title Language, dtype: int64

In [76]:
data_filtered['abstract'] = data_filtered['abstract'].apply(str)

In [77]:
language = []
for i in data_filtered['abstract']:
    language.append(detect(i))
data_filtered['Abstract Language']=language

In [78]:
data_filtered['Abstract Language'].value_counts()

en    11964
tl     5523
ro        1
ca        1
Name: Abstract Language, dtype: int64

In [44]:
type(data_filtered['abstract'])

pandas.core.series.Series

In [45]:
type(data_filtered['title'])

pandas.core.series.Series

In [46]:
type(data_filtered['Cleaned title'])

pandas.core.series.Series

In [79]:
filt = (data_filtered['Abstract Language'] =='en')
updated_df = data_filtered.loc[filt,data_filtered.columns]
updated_df.to_csv('./data2.csv', index = False)

In [80]:
d = pd.read_csv("data2.csv")

In [81]:
d.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,Title Language,Abstract Language
0,rr2wwvue,c64f525d77bf8b2ae4439a875f1bd8eb197a0519,PMC,Association and Host Selectivity in Multi-Host...,10.1371/journal.pone.0000041,PMC1762347,17183670.0,cc-by,The distribution of multi-host pathogens over ...,2006-12-20,...,PLoS One,NaN,NaN,NaN,document_parses/pdf_json/c64f525d77bf8b2ae4439...,document_parses/pmc_json/PMC1762347.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en,en
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,...,PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en,en
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,...,PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en,en
3,x2zq0lnt,d882f79283751d6c7f2e48cb991ee8abbcd913ae,PMC,The Transmissibility of Highly Pathogenic Avia...,10.1371/journal.pone.0000349,PMC1831494,17406673.0,cc-by,BACKGROUND: With the increased occurrence of o...,2007-04-04,...,PLoS One,NaN,NaN,NaN,document_parses/pdf_json/d882f79283751d6c7f2e4...,document_parses/pmc_json/PMC1831494.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en,en
4,s4y6uxsb,305cf1019ca9c81126b2e527fedf79969c2c522f,PMC,"ElaD, a Deubiquitinating Protease Expressed by...",10.1371/journal.pone.0000381,PMC1847702,17440617.0,cc-by,BACKGROUND: Ubiquitin and ubiquitin-like prote...,2007-04-18,...,PLoS One,NaN,NaN,NaN,document_parses/pdf_json/305cf1019ca9c81126b2e...,document_parses/pmc_json/PMC1847702.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,en,en


In [85]:
d['Title Language'].value_counts()

en    11813
it       57
ca       33
fr       16
ro       14
de        6
tl        4
af        3
nl        3
es        3
pt        2
so        2
fi        2
no        1
cy        1
et        1
id        1
tr        1
da        1
Name: Title Language, dtype: int64

### Named Recognisation Entity

In [86]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [103]:
## tag text
txt = d["abstract"].iloc[90]
doc = nlp(txt)
## display result
spacy.displacy.render(doc, style="ent")

In [98]:
def named_recognisation_entity(dtf,text):
## tag text and exctract tags into a list
    dtf["tags"] = dtf["text"].apply(lambda x: [(tag.text, tag.label_) 
                                for tag in ner(x).ents] )
## utils function to count the element of a list
    def utils_lst_count(lst):
        dic_counter = collections.Counter()
        for x in lst:
            dic_counter[x] += 1
            dic_counter = collections.OrderedDict( 
                     sorted(dic_counter.items(), 
                     key=lambda x: x[1], reverse=True))
        lst_count = [ {key:value} for key,value in dic_counter.items() ]
        return lst_count

## count tags
    dtf["tags"] = dtf["tags"].apply(lambda x: utils_lst_count(x))

## utils function create new column for each tag category
    def utils_ner_features(lst_dics_tuples, tag):
        if len(lst_dics_tuples) > 0:
            tag_type = []
            for dic_tuples in lst_dics_tuples:
                for tuple in dic_tuples:
                    type, n = tuple[1], dic_tuples[tuple]
                    tag_type = tag_type + [type]*n
                    dic_counter = collections.Counter()
                    for x in tag_type:
                        dic_counter[x] += 1
            return dic_counter[tag]
        else:
            return 0

## extract features
    tags_set = []
    for lst in dtf["tags"].tolist():
        for dic in lst:
            for k in dic.keys():
                tags_set.append(k[1])
    tags_set = list(set(tags_set))
    for feature in tags_set:
        dtf["tags_"+feature] = dtf["tags"].apply(lambda x: 
                             utils_ner_features(x, feature))


